# Computação Eficiente

Python é linguagem OO pensada como interface para sistemas de computação complexos. Assim, operações aparentemente simples escondem algoritmos sofisticados:

In [1]:
2+1

3

In [2]:
type(2)

int

In [3]:
(2).__add__(1)

3

_Observação_: use dir() para descobrir os atributos e métodos disponíveis para um objeto.

In [5]:
', '.join(dir(2))

'__abs__, __add__, __and__, __class__, __cmp__, __coerce__, __delattr__, __div__, __divmod__, __doc__, __float__, __floordiv__, __format__, __getattribute__, __getnewargs__, __hash__, __hex__, __index__, __init__, __int__, __invert__, __long__, __lshift__, __mod__, __mul__, __neg__, __new__, __nonzero__, __oct__, __or__, __pos__, __pow__, __radd__, __rand__, __rdiv__, __rdivmod__, __reduce__, __reduce_ex__, __repr__, __rfloordiv__, __rlshift__, __rmod__, __rmul__, __ror__, __rpow__, __rrshift__, __rshift__, __rsub__, __rtruediv__, __rxor__, __setattr__, __sizeof__, __str__, __sub__, __subclasshook__, __truediv__, __trunc__, __xor__, bit_length, conjugate, denominator, imag, numerator, real'

In [6]:
import numpy as np

In [5]:
v1 = np.random.rand(100)
v2 = np.random.rand(100)
v3 = v1 + v2
print v3

[0.75590693 1.6280265  0.28019357 1.28844675 1.74995991 1.56279259
 0.40533322 0.97511724 1.45848707 1.01912124 1.60712462 0.2140834
 0.33548886 0.38051654 1.16887429 1.34155249 1.23420732 0.60679364
 0.99980764 0.92729443 1.67092104 0.17870448 1.43959079 1.28311474
 1.64824349 0.93810819 0.59649105 0.87865905 0.6756044  0.74652094
 1.61813941 1.00223812 0.4341177  0.40113283 0.73572353 1.06743895
 0.66381358 1.43892241 0.82683549 0.68309313 1.60747984 0.73136176
 1.67093135 0.83918634 1.03177676 0.75937154 0.87071916 1.84073981
 1.61631459 1.02732814 0.9680996  1.25386684 0.40182916 0.3189567
 1.26591868 1.3798967  0.80923811 0.63086929 0.78451432 1.62779231
 0.50961288 1.14422077 0.15524168 1.09254776 0.69131687 0.94759213
 0.72048671 0.763398   1.07334944 1.6331252  0.84759556 1.24155087
 0.72452913 1.33612418 1.62590698 1.61480337 0.7618006  0.75947441
 0.85128812 1.00599919 0.50254406 1.2757516  0.22478639 0.53086427
 0.57837777 1.07010806 1.43536129 1.70683447 0.96362954 0.776749

In [6]:
m1 = np.random.rand(100, 15)
m2 = np.random.rand(15, 110)

In [7]:
def matmul(m1, m2):
    r = np.zeros((m1.shape[0], m2.shape[1]))
    for i in range(m1.shape[0]):
        for j in range(m2.shape[1]):
            for k in range(m2.shape[0]):
                r[i][j] += m1[i][k] * m2[k][j]
    return r

In [8]:
%timeit matmul(m1, m2)

10 loops, best of 3: 177 ms per loop


In [9]:
%timeit np.matmul(m1, m2)

The slowest run took 8.15 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 21.9 µs per loop


In [10]:
177000/21.4

8271.028037383177

Será que esta diferença de desempenho se deve ao uso do Python no meio... talvez não existisse se o código tivesse sido escrito em C e compilado de forma otimizada?

Comparação de multiplicação de matrizes em Numpy, Python (C-Python) e C

<img src="images/TLhDi" alt="NumpyVersusC" style="width: 500px;"/>

Por que, na comparação acima, o Numpy foi tão rápido? mais rápido mesmo que um código puro escrito em C? Basicamente, porque o código em numpy corresponde a versões otimizadas do melhor algotimo (prático) conhecido para o problema, escrito originalmente em Fortran para o BLAS. Abaixo, podemos ver a implementação do algoritmo de Strassen -- $O(n^{2.83})$, usado pelo numpy. Note o detalhe do _loop unrolling_ para acelerar o processamento em um hardware real.

```C

/*  -- translated by f2c (version 19940927).
   You must link the resulting object file with the libraries:
	-lf2c -lm   (in that order)
*/

#include "f2c.h"

doublereal sdot_(integer *n, real *sx, integer *incx, real *sy, integer *incy)
{


    /* System generated locals */
    integer i__1;
    real ret_val;

    /* Local variables */
    static integer i, m;
    static real stemp;
    static integer ix, iy, mp1;


/*     forms the dot product of two vectors.   
       uses unrolled loops for increments equal to one.   
       jack dongarra, linpack, 3/11/78.   
       modified 12/3/93, array(1) declarations changed to array(*)   


    
   Parameter adjustments   
       Function Body */
#define SY(I) sy[(I)-1]
#define SX(I) sx[(I)-1]


    stemp = 0.f;
    ret_val = 0.f;
    if (*n <= 0) {
	return ret_val;
    }
    if (*incx == 1 && *incy == 1) {
	goto L20;
    }

/*        code for unequal increments or equal increments   
            not equal to 1 */

    ix = 1;
    iy = 1;
    if (*incx < 0) {
	ix = (-(*n) + 1) * *incx + 1;
    }
    if (*incy < 0) {
	iy = (-(*n) + 1) * *incy + 1;
    }
    i__1 = *n;
    for (i = 1; i <= *n; ++i) {
	stemp += SX(ix) * SY(iy);
	ix += *incx;
	iy += *incy;
/* L10: */
    }
    ret_val = stemp;
    return ret_val;

/*        code for both increments equal to 1   


          clean-up loop */

L20:
    m = *n % 5;
    if (m == 0) {
	goto L40;
    }
    i__1 = m;
    for (i = 1; i <= m; ++i) {
	stemp += SX(i) * SY(i);
/* L30: */
    }
    if (*n < 5) {
	goto L60;
    }
L40:
    mp1 = m + 1;
    i__1 = *n;
    for (i = mp1; i <= *n; i += 5) {
	stemp = stemp + SX(i) * SY(i) + SX(i + 1) * SY(i + 1) + SX(i + 2) * 
		SY(i + 2) + SX(i + 3) * SY(i + 3) + SX(i + 4) * SY(i + 4);
/* L50: */
    }
L60:
    ret_val = stemp;
    return ret_val;
} /* sdot_ */

```

### O Tensorflow é o numpy para máquinas vetorias

Contudo, usa uma abordagem estática. Ou seja, o grafo de computação é declarado primeiro e, então, compilado e executado.

In [11]:
import tensorflow as tf

/Users/marcocristo/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [12]:
t1 = tf.constant(np.random.rand(100, 15), dtype = tf.float32)
t2 = tf.constant(np.random.rand(15, 110), dtype = tf.float32)

In [13]:
t3 = tf.matmul(t1, t2)

In [14]:
t3

<tf.Tensor 'MatMul:0' shape=(100, 110) dtype=float32>

In [15]:
with tf.Session() as s:
    print s.run(t3)

[[3.8121727 4.6881065 3.8925529 ... 4.304786  4.610894  5.0712986]
 [2.2580144 3.3412476 2.781353  ... 3.7897735 4.6492434 4.1774445]
 [3.5686884 2.8873925 2.310885  ... 3.447249  3.0482647 3.0317369]
 ...
 [2.7807617 3.2592428 3.3020627 ... 4.0484824 3.4263897 3.6995542]
 [3.2312806 3.0449948 2.7674925 ... 3.6788912 3.325767  3.54379  ]
 [2.9236672 3.6910748 4.350985  ... 3.8461893 4.007058  4.39439  ]]


In [16]:
def sigmoid(t):
    return 1.0 / (1.0 + tf.exp(-t))

In [17]:
t1 = tf.constant(np.random.rand(100, 15), dtype = tf.float32)
t2 = tf.constant(np.random.rand(15, 110), dtype = tf.float32)

In [18]:
t1s = sigmoid(t1)

In [19]:
t1s

<tf.Tensor 'div:0' shape=(100, 15) dtype=float32>

In [20]:
with tf.Session() as s:
    print s.run(t1s)

[[0.649144   0.70794886 0.5168201  ... 0.6655356  0.63889915 0.65131783]
 [0.55800515 0.5566466  0.5325955  ... 0.6143507  0.5686545  0.7058372 ]
 [0.6440298  0.72477514 0.5266039  ... 0.69094425 0.5204311  0.71589535]
 ...
 [0.5660618  0.6058463  0.5939491  ... 0.5635986  0.5615751  0.5925196 ]
 [0.7003494  0.7308743  0.7261002  ... 0.7249049  0.639606   0.56627923]
 [0.638626   0.65363836 0.53531146 ... 0.62159365 0.6410436  0.7244281 ]]


In [21]:
t1 = tf.placeholder(shape = (100, 15), dtype = tf.float32)
t2 = tf.placeholder(shape = (15, 110), dtype = tf.float32)

In [22]:
t3 = tf.matmul(t1, t2)

In [23]:
t3

<tf.Tensor 'MatMul_1:0' shape=(100, 110) dtype=float32>

In [25]:
with tf.Session() as s:
    print s.run(t3, feed_dict = {t1: m1, t2: m2})
    m1 = 2 * m1
    m2 = 3 * m2
    print s.run(t3, feed_dict = {t1: m1, t2: m2})

[[3.0664072 3.7516713 3.9476361 ... 5.061258  3.193798  4.9117174]
 [2.6090896 3.307047  3.7638092 ... 4.215127  2.5441265 4.655652 ]
 [3.4270198 3.5554903 4.8707795 ... 4.109871  3.2171454 4.139707 ]
 ...
 [2.7442398 3.115177  3.6109269 ... 3.7556608 2.4192553 3.322314 ]
 [2.80262   2.5835788 4.2646103 ... 3.5568447 2.84881   4.2548866]
 [3.4731297 3.3111138 5.0291147 ... 4.1220503 3.4891956 4.332391 ]]
[[18.398443 22.510027 23.685816 ... 30.367548 19.162788 29.470306]
 [15.654536 19.842278 22.582853 ... 25.290764 15.264757 27.933912]
 [20.562119 21.332941 29.224678 ... 24.659225 19.302872 24.838245]
 ...
 [16.465435 18.691061 21.665562 ... 22.53396  14.515531 19.933882]
 [16.81572  15.501473 25.58766  ... 21.341068 17.092857 25.529322]
 [20.838778 19.866684 30.174686 ... 24.732304 20.935175 25.994349]]


O problema com essa abordagem é que ela ainda é relativamente baixo nível. Logo, não é produtiva em termos de prototipação. Assim, surge a necessidade de APIs de mais alto nível ainda, como o Keras.